# Calculating GB energy from DF

## Objective

(i) Select data from boundaries
(ii) Organize it for all grains
(iii) Calculate the energy with wield

## Method

    * Open Dataframe
    * Select attributes of interest from boundaries
    * Create a new DF with grain information
    * Convert data to wield
    * Run wield for all grains
    * Store energy in Dataframe

In [ ]:
import pandas as pd
import numpy as np
from skimage import draw,io
from skimage.segmentation import flood, flood_fill
from scipy.spatial import ConvexHull, convex_hull_plot_2d
from matplotlib import pyplot as plt
from IPython.display import set_matplotlib_formats
plt.rcParams['figure.dpi'] = 200
plt.rcParams['savefig.dpi'] = 200


### Open Dataframe


In [ ]:

folder = "../data/"
file = "1_005"
path = folder + file

#%%

sample = np.loadtxt(path+ ".txt")

'''
# Column 1-3:   right hand average orientation (phi1, PHI, phi2 in radians)
# Column 4-6:   left hand average orientation (phi1, PHI, phi2 in radians)
# Column 7:     Misorientation Angle
# Column 8-10:  Misorientation Axis in Right Hand grain
# Column 11-13: Misorientation Axis in Left Hand grain
# Column 14:    length (in microns)
# Column 15:    trace angle (in degrees)
# Column 16-19: x,y coordinates of endpoints (in microns)
# Column 20-21: IDs of right hand and left hand grains

'''


df = pd.DataFrame(  data = sample, 
                    columns = ["right_phi1","right_PHI","right_phi2",                 #1-3
                               "left_phi1","left_PHI","left_phi2",                    #4-6 
                               "ori_angle",                                           #7
                               "right_ori_x","right_ori_y","right_ori_z",              #8-10
                               "left_ori_x","left_ori_y","left_ori_z",                 #11-13  
                               "length",                                              #14
                               "trace_angle",                                         #15
                               "x_start", "y_start", "x_end", "y_end",                #16-19
                               "grain_right","grain_left"                             #20-21
                               ]                    
                 )


In [ ]:
df.grain_right.drop_duplicates(keep = False)

### Select attributes of interest from boundaries

The group of information that is directly related to the grain: 
* Column 1-3:   right hand average orientation (phi1, PHI, phi2 in radians)
* Column 4-6:   left hand average orientation (phi1, PHI, phi2 in radians)
* Column 7:     Misorientation Angle
* Column 8-10:  Misorientation Axis in Right Hand grain
* Column 11-13: Misorientation Axis in Left Hand grain
* Column 20-21: IDs of right hand and left hand grains

The Columns 20-21 describe the grain of interest

The Columns 1-6 depends on grain itself, **here we are interested**

The Columns 7-13 depends on left and right grains

In [ ]:
#df_left = df[['ori_angle','left_phi1','left_PHI','left_phi2','left_ori_x','left_ori_y','left_ori_z','grain_left']]
#df_right = d|f[['right_phi1','right_PHI','right_phi2','right_ori_x','right_ori_y','right_ori_z','grain_right']]

# left_ori is related with other grain, than is not interesting to keep in this case

df_left = df[['left_phi1','left_PHI','left_phi2','grain_left']]
df_left = df_left.rename(columns={"grain_left": "grain"})


df_right = df[['right_phi1','right_PHI','right_phi2','grain_right']]
df_right = df_right.rename(columns={"grain_right": "grain"})


In [ ]:
df_left = df_left[~df_left.grain.duplicated()].sort_values('grain')
df_left = df_left.set_index('grain')


In [ ]:
df_right = df_right[~df_right.grain.duplicated()].sort_values('grain')
df_right = df_right.set_index('grain')


In [ ]:
#df_grains = pd.concat([df_left,df_right])
df_grains = df_left.join(df_right)
df_grains_l = df_left.join(df_right)
df_grains_r = df_right.join(df_left)

In [ ]:
df_grains_l.isnull().sum(),df_grains_r.isnull().sum()

In [ ]:
df_grains.sort_values('grain')

In [ ]:
df_grains.isnull().sum()

## Fill Grains based on phi1, PHI, and phi2

In [ ]:
df_grains_norm = (df_grains - df_grains.min()) / (df_grains.max() - df_grains.min())

In [ ]:
grain = 1512

One_grain = df[(df["grain_right"] == grain) | (df["grain_left"] == grain)]
One_grain

In [ ]:
#del mask
grain_info = df_grains_norm.loc[grain,:]
grain_info

In [ ]:

width = int(max([max(One_grain.x_end),max(One_grain.x_start)]))+1
height = int(max([max(One_grain.y_end),max(One_grain.y_start)]))+1

width,height

In [ ]:
x_center = 1000#int(One_grain[['x_start','x_end']].mean().mean())
y_center = 300#int(One_grain[['y_start','y_end']].mean().mean())

In [ ]:
One_grain.y_start.iloc[1:3]

The method polygon_fill do not work because it uses the Even–odd rule and fill all closed paths based on directions provided as parameters. As the order or parameters meters and in our case is not ordered, this method is not helpful.


## Flood Fill method

In [ ]:
One_grain.x_start

In [ ]:

np_img = np.zeros([height, width, 3])
np_img2 = np.zeros([height, width, 3])

for idx, row in One_grain.iterrows():
    #print("%d %d %d %d\n" %(row.x_start.astype("uint16"),row.y_start.astype("uint16"),row.x_end.astype("uint16"),row.y_end.astype("uint16")))
    
    rr,cc,a = draw.line_aa(row.x_start.astype("uint16"),row.y_start.astype("uint16"),row.x_end.astype("uint16"),row.y_end.astype("uint16"))
    np_img[cc,rr,:3] = (1,1,1)
    
#mask = flood_fill(np_img, (50, 52,1), 255 )
mask = flood(np_img, (y_center, x_center,0))
#np_img = flood_fill(np_img, (50, 52,0), 155 )
np.shape(mask[:,:,1])

plt.imshow(np_img)

In [ ]:
phi1,Phi,phi2 = grain_info[["right_phi1","right_PHI","right_phi2"]]
np_img2[mask[:,:,1]] = [phi1,Phi,phi2]
np_img2[np_img[:,:,1] !=0] =  [phi1,Phi,phi2]


plt.imshow(np_img2)

In [ ]:
grain = 1512

One_grain = df[(df["grain_right"] == grain) | (df["grain_left"] == grain)]
grain_info = df_grains_norm.loc[grain,:]
x_center = int(One_grain[['x_start','x_end']].mean().mean())
y_center = int(One_grain[['y_start','y_end']].mean().mean())

np_img = np.zeros([height, width, 3])
#np_img2 = np.zeros([height, width, 3])

for idx, row in One_grain.iterrows():
    #print("%d %d %d %d\n" %(row.x_start.astype("uint16"),row.y_start.astype("uint16"),row.x_end.astype("uint16"),row.y_end.astype("uint16")))
    
    rr,cc,a = draw.line_aa(row.x_start.astype("uint16"),row.y_start.astype("uint16"),row.x_end.astype("uint16"),row.y_end.astype("uint16"))
    np_img[cc,rr,:3] = (1,1,1)
    
    
#mask = flood_fill(np_img, (50, 52,1), 255 )
mask = flood(np_img, (y_center, x_center,0))
#np_img = flood_fill(np_img, (50, 52,0), 155 )
np.shape(mask[:,:,1])

plt.imshow(np_img)

In [ ]:
phi1,Phi,phi2 = grain_info[["right_phi1","right_PHI","right_phi2"]]
np_img[np_img[:,:,1] !=0] =  [phi1,Phi,phi2]
np_img[mask[:,:,1] !=0] = [phi1,Phi,phi2]
np_img2 += np_img

In [ ]:
plt.imshow(np_img)

In [ ]:
plt.imshow(np_img2)

Applying to all grains

In [ ]:

import cv2
import time
import math



df_grains_norm = (df_grains - df_grains.min()) / (df_grains.max() - df_grains.min())

width = int(max([max(df.x_end),max(df.x_start)]))+1
height = int(max([max(df.y_end),max(df.y_start)]))+1

#for idx, row in df.iterrows():
#    rr,cc,a = draw.line_aa(row.x_start.astype("uint16"),row.y_start.astype("uint16"),row.x_end.astype("uint16"),row.y_end.astype("uint16"))
#    np_img[cc,rr,:3] = (1,1,1)
    
#plt.imshow(np_img)


In [ ]:
from multiprocessing import Process
 
# execute a task
def task(grain):
    global flood_grains
    overflood = np.sum(flood_grains==0) * 0.8
    One_grain = df[(df["grain_right"] == grain) | (df["grain_left"] == grain)]
    grain_info = df_grains_norm.loc[grain,:]
    np_img = np.zeros([height, width, 3])

  #  width = int(max([max(One_grain.x_end),max(One_grain.x_start)]))+1
  #  height = int(max([max(One_grain.y_end),max(One_grain.y_start)]))+1

    x_center = math.floor(One_grain[['x_start','x_end']].mean().mean())
    y_center = math.floor(One_grain[['y_start','y_end']].mean().mean())


    if(x_center < 200 and y_center < 200):    
        phi1,Phi,phi2 = grain_info[["right_phi1","right_PHI","right_phi2"]]
        #    cv2.putText(np_img, text=str(int(grain)), org=(x_center,y_center),fontFace=2, fontScale=0.4, color=(255,255,255), thickness=1)
    
        for idx, row in One_grain.iterrows():
            
            rr,cc,a = draw.line_aa(row.x_start.astype("uint16"),row.y_start.astype("uint16"),row.x_end.astype("uint16"),row.y_end.astype("uint16"))
            np_img[cc,rr] = (1,1,1)
        
        mask = flood(np_img, (y_center, x_center,0))
#        print(str(grain) + " len "+ str(np.count_nonzero(mask)))
#        print(str(grain) + " len 0 "+ str(np.sum(mask==1)))
        np_img[np_img[:,:,1] !=0] =  [0,0,0]#[phi1,Phi,phi2]
        


        if (np.sum(mask==1)<overflood):
            np_img[mask[:,:,1] !=0] = [phi1,Phi,phi2]
            
        else:
            start = pd.DataFrame(columns=["x","y"])
            end = pd.DataFrame(columns=["x","y"])
            start[["x","y"]] = One_grain[['x_start','y_start']]
            end[["x","y"]] = One_grain[['x_end','y_end']]
            points = pd.concat([start,end])
            

            
            try :
                hull = ConvexHull(points)
                for idx in range(len(hull.simplices)):
                    x_s = hull.points[hull.simplices[idx,0], 0].astype("uint16")
                    y_s = hull.points[hull.simplices[idx,0], 1].astype("uint16")
                    x_e = hull.points[hull.simplices[idx,1], 0].astype("uint16")
                    y_e = hull.points[hull.simplices[idx,1], 1].astype("uint16")

                    rr,cc,a = draw.line_aa(x_s,y_s,x_e,y_e)
                    
                    np_img[cc,rr] = (1,1,1)
                    mask = flood(np_img, (y_center, x_center,0))

                    np_img[np_img[:,:,1] !=0] =  [0,0,0]
                    np_img[mask[:,:,1] !=0] = [phi1,Phi,phi2]
            except:
#                 print(points)
                pass
           
            

            
        flood_grains += np_img
#         cv2.imshow('NP',flood_grains)
#         cv2.waitKey(0)
#         cv2.destroyAllWindows()
        return np_img


In [ ]:
plt.imshow(flood_grains)

In [ ]:
%%time

width = int(max([max(df.x_end),max(df.x_start)]))+1
height = int(max([max(df.y_end),max(df.y_start)]))+1
flood_grains = np.zeros([height, width, 3])
overflood = np.sum(flood_grains==0) * 0.8

for grain in df_grains.index:
    One_grain = df[(df["grain_right"] == grain) | (df["grain_left"] == grain)]
    grain_info = df_grains_norm.loc[grain,:]
    np_img = np.zeros([height, width, 3])

  #  width = int(max([max(One_grain.x_end),max(One_grain.x_start)]))+1
  #  height = int(max([max(One_grain.y_end),max(One_grain.y_start)]))+1

    x_center = math.floor(One_grain[['x_start','x_end']].mean().mean())
    y_center = math.floor(One_grain[['y_start','y_end']].mean().mean())


    if(x_center < 200 and y_center < 200): 
        phi1,Phi,phi2 = grain_info[["right_phi1","right_PHI","right_phi2"]]
        #    cv2.putText(np_img, text=str(int(grain)), org=(x_center,y_center),fontFace=2, fontScale=0.4, color=(255,255,255), thickness=1)
    
        for idx, row in One_grain.iterrows():
            
            rr,cc,a = draw.line_aa(row.x_start.astype("uint16"),row.y_start.astype("uint16"),row.x_end.astype("uint16"),row.y_end.astype("uint16"))
            np_img[cc,rr] = (1,1,1)
        
        mask = flood(np_img, (y_center, x_center,0))
#        print(str(grain) + " len "+ str(np.count_nonzero(mask)))
#        print(str(grain) + " len 0 "+ str(np.sum(mask==1)))
        np_img[np_img[:,:,1] !=0] =  [0,0,0]#[phi1,Phi,phi2]
        
        cv2.imshow('NP',np_img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

        if (np.sum(mask==1)<overflood):
            np_img[mask[:,:,1] !=0] = [phi1,Phi,phi2]
            print("here")
            flood_grains += np_img
            
        else:
            start = pd.DataFrame(columns=["x","y"])
            end = pd.DataFrame(columns=["x","y"])
            start[["x","y"]] = One_grain[['x_start','y_start']]
            end[["x","y"]] = One_grain[['x_end','y_end']]
            points = pd.concat([start,end])
            

            
            try :
                hull = ConvexHull(points)
                for idx in range(len(hull.simplices)):
                    x_s = hull.points[hull.simplices[idx,0], 0].astype("uint16")
                    y_s = hull.points[hull.simplices[idx,0], 1].astype("uint16")
                    x_e = hull.points[hull.simplices[idx,1], 0].astype("uint16")
                    y_e = hull.points[hull.simplices[idx,1], 1].astype("uint16")

                    rr,cc,a = draw.line_aa(x_s,y_s,x_e,y_e)
                    
                    np_img[cc,rr] = (1,1,1)
                    mask = flood(np_img, (y_center, x_center,0))

                    np_img[np_img[:,:,1] !=0] =  [0,0,0]
                    np_img[mask[:,:,1] !=0] = [phi1,Phi,phi2]
                    flood_grains += np_img
                
            except:
           #     print(points)
                pass
           
        
#             cv2.imshow('Image',np_img)
#             cv2.waitKey(0)
#             cv2.destroyAllWindows()

            
        



plt.imshow(flood_grains)
#io.imsave("processing/"+ file +'.png',np_img)

In [ ]:
cv2.imshow('F',flood_grains)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
io.imsave("processing/"+ file +'.png',flood_grains)

In [ ]:
grain = 1512
One_grain = df[(df["grain_right"] == grain) | (df["grain_left"] == grain)]


width = int(max([max(df.x_end),max(df.x_start)]))+1
height = int(max([max(df.y_end),max(df.y_start)]))+1


grain_info = df_grains_norm.loc[grain,:]
np_img = np.zeros([height, width, 3])


x_center = math.floor(One_grain[['x_start','x_end']].mean().mean())
y_center = math.floor(One_grain[['y_start','y_end']].mean().mean())


if(x_center <2100 and y_center < 2100):    
    phi1,Phi,phi2 = grain_info[["right_phi1","right_PHI","right_phi2"]]
    #    cv2.putText(np_img, text=str(int(grain)), org=(x_center,y_center),fontFace=2, fontScale=0.4, color=(255,255,255), thickness=1)

    start = pd.DataFrame(columns=["x","y"])
    end = pd.DataFrame(columns=["x","y"])
    start[["x","y"]] = One_grain[['x_start','y_start']]
    end[["x","y"]] = One_grain[['x_end','y_end']]
    points = pd.concat([start,end])
    points.reset_index()



    hull = ConvexHull(points)
    for idx in range(len(hull.simplices)):
        x_s = hull.points[hull.simplices[idx,0], 0].astype("uint16")
        y_s = hull.points[hull.simplices[idx,0], 1].astype("uint16")
        x_e = hull.points[hull.simplices[idx,1], 0].astype("uint16")
        y_e = hull.points[hull.simplices[idx,1], 1].astype("uint16")

        rr,cc,a = draw.line_aa(x_s,y_s,x_e,y_e)
        np_img[cc,rr] = (1,1,1)

    mask = flood(np_img, (y_center, x_center,0))
    
    np_img[np_img[:,:,1] !=0] =  [phi1,Phi,phi2]
    np_img[mask[:,:,1] !=0] = [phi1,Phi,phi2]
    
plt.imshow(np_img)
print(np.sum(mask==1))

In [ ]:

full_img = np_img

for idx, row in df.iterrows():
    #print("%d %d %d %d\n" %(row.x_start.astype("uint16"),row.y_start.astype("uint16"),row.x_end.astype("uint16"),row.y_end.astype("uint16")))    
    rr,cc= draw.line(row.x_start.astype("uint16"),row.y_start.astype("uint16"),row.x_end.astype("uint16"),row.y_end.astype("uint16"))
    full_img[cc,rr,:3] = (1,1,1)

plt.imshow(full_img)

In [ ]:
cv2.imshow('f',flood_grains)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
io.imsave("processing/"+ file +'_partial.png',np_img)

Some grains are not in the df_grains dataframe.

In [ ]:
df[df["grain_left"].isnull()]

In [ ]:
df["grain_left"].unique()

In [ ]:
for a in df["grain_right"].unique():
    for b in df["grain_left"].unique():
        if a == b:
          # print("right found " + str(a))
            break
        if b == df["grain_left"].unique()[-1]:
            print("right not found " + str(a))

In [ ]:
for a in df["grain_left"].unique():
    for b in df["grain_right"].unique():
        if a == b:
          # print("right found " + str(a))
            break
        if b == df["grain_right"].unique()[-1]:
            print("left not found " + str(a))

## Closing opened grains

In [ ]:

grain = 1512


One_grain = df[(df["grain_right"] == grain) | (df["grain_left"] == grain)]


grain_info = df_grains_norm.loc[grain,:]
np_img = np.zeros([height, width, 3])

start = pd.DataFrame(columns=["x","y"])
end = pd.DataFrame(columns=["x","y"])
start[["x","y"]] = One_grain[['x_start','y_start']]
end[["x","y"]] = One_grain[['x_end','y_end']]
points = pd.concat([start,end])
#points.reset_index()

    
points

In [ ]:
One_grain[['x_start','y_start','x_end','y_end']]

In [ ]:
points.drop_duplicates(keep=False).astype('int32')

In [ ]:
img = np.zeros([height+1,width+1, 3])

#rr,cc = draw.line(int(One_grain.iloc[0].x_start),int(One_grain.iloc[0].y_start),int(One_grain.iloc[-1].x_end),int(One_grain.iloc[-1].y_end))
#img[cc,rr] = (255,255,0)
    
for idx, row in One_grain[["x_start","y_start","x_end","y_end"]].astype('int32').iterrows():
    rr,cc = draw.line(row.x_start,row.y_start,row.x_end,row.y_end)

    img[cc,rr] = (0,200,0)

plt.imshow(img[350:400,1070:1150])  

In [ ]:


width = int(max([max(df.x_end),max(One_grain.x_start)]))+1
height = int(max([max(One_grain.y_end),max(One_grain.y_start)]))+1
np_img = np.zeros([height, width, 3])

hull = ConvexHull(points)
for idx in range(len(hull.simplices)):
    x_s = hull.points[hull.simplices[idx,0], 0].astype("uint16")
    y_s = hull.points[hull.simplices[idx,0], 1].astype("uint16")
    x_e = hull.points[hull.simplices[idx,1], 0].astype("uint16")
    y_e = hull.points[hull.simplices[idx,1], 1].astype("uint16")
    
    rr,cc,a = draw.line_aa(x_s,y_s,x_e,y_e)
    np_img[cc,rr] = (1,1,1)


In [ ]:
plt.imshow(np_img)

In [ ]:
len(hull.simplices)

In [ ]:
hull = ConvexHull(points)
a = convex_hull_plot_2d(hull)
plt.show()


In [ ]:
import cv2
np_img = np.zeros([50, 50, 3])
test = cv2.fillPoly(np_img, pts = [points.values.tolist()], color =(255,255,255))
plt.imshow(test)

In [ ]:
hull.points[hull.simplices[0,1], :] 

# New Method to draw and fill

### Order points in clockwise direction

In [ ]:
import math

origin = [0, 0]
refvec = [0, 1]

def clockwiseangle_and_distance(point):
    # Vector between point and the origin: v = p - o
    vector = [point[0]-origin[0], point[1]-origin[1]]
    
    # Length of vector: ||v||
    lenvector = math.hypot(vector[0], vector[1])
   
    # If length is zero there is no angle
    if lenvector == 0:
        return -math.pi, 0
    # Normalize vector: v/||v||
    normalized = [vector[0]/lenvector, vector[1]/lenvector]
    dotprod  = normalized[0]*refvec[0] + normalized[1]*refvec[1]     # x1*x2 + y1*y2
    diffprod = refvec[1]*normalized[0] - refvec[0]*normalized[1]     # x1*y2 - y1*x2
    angle = math.atan2(diffprod, dotprod)
    # Negative angles represent counter-clockwise angles so we need to subtract them 
    # from 2*pi (360 degrees)
    if angle < 0:
        return 2*math.pi+angle, lenvector
    # I return first the angle because that's the primary sorting criterium
    # but if two vectors have the same angle then the shorter distance should come first.
    return angle, lenvector

In [ ]:
p1

In [ ]:
One_grain

In [ ]:


width = int(max([max(df.x_end),max(df.x_start)]))+1
height = int(max([max(df.y_end),max(df.y_start)]))+1

flood_grains = np.zeros([height, width, 3])
overflood = np.sum(flood_grains==0) * 0.8
print(overflood)


#for grain in df_grains.index:
grain = 1512
One_grain = df[(df["grain_right"] == grain) | (df["grain_left"] == grain)]
grain_info = df_grains_norm.loc[grain,:]
np_img = np.zeros([height, width, 3])

#  width = int(max([max(One_grain.x_end),max(One_grain.x_start)]))+1
#  height = int(max([max(One_grain.y_end),max(One_grain.y_start)]))+1

x_center = math.floor(One_grain[['x_start','x_end']].mean().mean())
y_center = math.floor(One_grain[['y_start','y_end']].mean().mean())


if(x_center < 200 and y_center < 200): 
    phi1,Phi,phi2 = grain_info[["right_phi1","right_PHI","right_phi2"]]
    #    cv2.putText(np_img, text=str(int(grain)), org=(x_center,y_center),fontFace=2, fontScale=0.4, color=(255,255,255), thickness=1)

    for idx, row in One_grain.iterrows():

        rr,cc,a = draw.line_aa(row.x_start.astype("uint16"),row.y_start.astype("uint16"),row.x_end.astype("uint16"),row.y_end.astype("uint16"))
        np_img[cc,rr] = (1,1,1)

    mask = flood(np_img, (y_center, x_center,0))
#        print(str(grain) + " len "+ str(np.count_nonzero(mask)))
#        print(str(grain) + " len 0 "+ str(np.sum(mask==1)))
    np_img[np_img[:,:,1] !=0] =  [0,0,0]#[phi1,Phi,phi2]

    if (np.sum(mask==1)<overflood):
        np_img[mask[:,:,1] !=0] = [phi1,Phi,phi2]
       # print("Grain" + str(grain))


    else:

        start = pd.DataFrame(columns=["x","y"])
        end = pd.DataFrame(columns=["x","y"])
        start[["x","y"]] = One_grain[['x_start','y_start']]
        end[["x","y"]] = One_grain[['x_end','y_end']]
        points = pd.concat([start,end])

        p = points.drop_duplicates()
        p1 = p.to_numpy()

        origin = p1[0]
        sort = sorted(p1, key=clockwiseangle_and_distance)
        a = []
        for b in sort:
            a.append(tuple((int(b[0]),int(b[1]))))

        cv2.polylines(np_img, np.array([a]), True, (255,255,255), 1)

        mask = flood(np_img, (y_center, x_center,0))

    flood_grains += np_img
plt.imshow(np_img)#[340:400,1050:1150])

cv2.imshow('f',np_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
sort = sorted(p1, key=clockwiseangle_and_distance)
a = []
for b in sort:
    a.append(tuple((int(b[0]),int(b[1]))))

In [ ]:
# pts = [(50, 50), (300, 190), (400, 10)]
# pt = [(1, 1150), (2, 1154), (3, 1194),(1, 1153),(1, 1160), (1, 1173),(1, 1180),(1, 1184), (1, 1188)]
# np.shape(pt)

In [ ]:
import cv2





